# Debugging A Simple Image Processing Program Using The Compute Sanitizer
## 04 Memcheck part 2

In this section, we will continue on the memcheck tool to explore a new option
that was added with the compute-sanitizer version released with CUDA 11.3.

In the current state of the program, as we saw previously something is still off
as in the left part of the image some pixels are not colored correctly. Let's try to run memcheck again:


```
========= COMPUTE-SANITIZER
Image ./checkerboard.png size: 32 * 32
Image ./icon.png size: 32 * 32
Grid size: 2 * 2 * 1
Block size: 31 * 31 * 1
Grid size: 2 * 2 * 1
Block size: 31 * 31 * 1
========= ERROR SUMMARY: 0 errors

```

No errors are reported. At this point we could have several hypothesis:

- The program doesn't do any undefined behavior and the problem is purely in its
  logic, in which case, using the CUDA debugger would be a better option for
  debugging.
- The program has an undefined behavior that could be detected by another tool,
  or that the compute-sanitizer cannot help with
- The program does out of bound accesses, but ends up in a
  neighboring allocation, which makes the access seen as valid while it would
  not be.
- ...

Indeed, for the latter point, depending on the size of the allocations and their
alignment, allocations can be directly consecutive in the virtual address space,
meaning that going out of bound of a first allocation might result in an access
in the following allocation which is seen as valid. As a countermeasure to this
issue, the compute-sanitizer introduced the flag `--padding` which makes the
compute-sanitizer add some padding bytes between allocations in order to avoid
said consecutive allocation and better detect out of bound accesses in such
case. This can have further impact on the memory consumption and performances of
the program and is therefore not enabled by default.
Let's see if that flag can help us:


```sh
    compute-sanitizer --padding=1 ./convolution
```

We observe this output:


```
========= COMPUTE-SANITIZER
Image ./checkerboard.png size: 32 * 32
Image ./icon.png size: 32 * 32
Grid size: 2 * 2 * 1
Block size: 31 * 31 * 1
========= Invalid __global__ write of size 1 bytes
=========     at 0x7800 in /home/nvidia/sanitizer-demo/convolution.cu:135:convolution(pixel const*, int, int, float const*, int, pixel*)
=========     by thread (1,0,0) in block (1,1,0)
=========     Address 0x7f1397001003 is out of bounds
=========     Saved host backtrace up to driver entry point at kernel launch time
=========     Host Frame: [0x27e53a]
=========                in /usr/lib/x86_64-linux-gnu/libcuda.so.1
=========     Host Frame: [0x1053b]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x5bf60]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xae3e]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__device_stub__Z11convolutionPK5pixeliiPKfiPS_(pixel const *, int, int, float const *, int, pixel*) [0xac7b]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:convolution(pixel const *, int, int, float const *, int, pixel*) [0xace8]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xa298]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:main [0xa7ca]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__libc_start_main [0x270b3]
=========                in /lib/x86_64-linux-gnu/libc.so.6
=========     Host Frame:_start [0x9c0e]
=========                in /home/nvidia/sanitizer-demo/./convolution
========= 
========= Program hit unspecified launch failure (error 719) on CUDA API call to cudaDeviceSynchronize.
=========     Saved host backtrace up to driver entry point at error
=========     Host Frame: [0x54b673]
=========                in /usr/lib/x86_64-linux-gnu/libcuda.so.1
=========     Host Frame: [0x3cbc7]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xa2ba]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:main [0xa7ca]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__libc_start_main [0x270b3]
=========                in /lib/x86_64-linux-gnu/libc.so.6
=========     Host Frame:_start [0x9c0e]
=========                in /home/nvidia/sanitizer-demo/./convolution
========= 
GPU error: unspecified launch failure at convolution.cu:183
========= Target application returned an error
========= ERROR SUMMARY: 2 errors

```

We can now see errors that were previously undetected. Using this new
information, take a moment to try to find the error. The solution is in the following collapsed cell:

The error is located at line 110, where `xIdx > width` should be `xIdx >= width`, as the x index equal to width would be out of bounds by one.

Fix the mistake, recompile and run the program again. If we are lucky, we should see the correct output for [checkerboard-out.png](code/checkerboard-out.png) and [icon-out.png](code/icon-out.png), but this is not a guarantee as there is still a problem remaining in the code.

[Continue to step 05.](05_racecheck.ipynb)